In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth


dataset = [
    ["Milk", "Onion", "Nutmeg", "Kidney Beans", "Eggs", "Yogurt"],
    ["Dill", "Onion", "Nutmeg", "Kidney Beans", "Eggs", "Yogurt"],
    ["Milk", "Apple", "Kidney Beans", "Eggs"],
    ["Milk", "Unicorn", "Corn", "Kidney Beans", "Yogurt"],
    ["Corn", "Onion", "Onion", "Kidney Beans", "Ice cream", "Eggs"],
]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(df, min_support=0.6, use_colnames=True)
### alternatively:
# frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
# frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

frequent_itemsets


,support,itemsets
0,1.0,(Kidney Beans)
1,0.8,(Eggs)
2,0.6,(Yogurt)
3,0.6,(Onion)
4,0.6,(Milk)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Yogurt, Kidney Beans)"
7,0.6,"(Onion, Eggs)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Onion, Kidney Beans, Eggs)"


In [2]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.0,0.0
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf,0.0
2,(Yogurt),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
3,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5
4,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1.0
5,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
6,"(Onion, Kidney Beans)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5
7,"(Onion, Eggs)",(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
8,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1.0
9,(Onion),"(Kidney Beans, Eggs)",0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5


In [3]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5
1,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1.0
2,"(Onion, Kidney Beans)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5
3,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1.0
4,(Onion),"(Kidney Beans, Eggs)",0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5
5,(Eggs),"(Onion, Kidney Beans)",0.8,0.6,0.6,0.75,1.25,0.12,1.6,1.0


In [5]:
rules[
    (rules["antecedent_len"] >= 2)
    & (rules["confidence"] > 0.75)
    & (rules["lift"] > 1.2)
]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len
2,"(Onion, Kidney Beans)",(Eggs),0.6,0.8,0.6,1.0,1.25,0.12,inf,0.5,2


In [6]:
df1 = pd.read_csv("sales.csv", usecols=["order_id", "category"])

/tmp/ipykernel_305647/3577836999.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("sales.csv", usecols=["order_id", "category"])


In [8]:
df1.head()

,order_id,category
0,100354678,Men's Fashion
1,100354678,Men's Fashion
2,100354680,Men's Fashion
3,100354680,Men's Fashion
4,100367357,Men's Fashion


In [16]:
# Group by 'order_id' and aggregate 'category' values into a list
grouped = df1.groupby("order_id")["category"].apply(list).reset_index()

# Convert the grouped DataFrame to the desired list format
dataset = grouped["category"].tolist()


In [ ]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
# , min_support=0.1
frequent_itemsets = fpgrowth(df, use_colnames=True)

In [24]:
df

,Appliances,Beauty & Grooming,Books,Computing,Entertainment,Health & Sports,Home & Living,Kids & Baby,Men's Fashion,Mobiles & Tablets,Others,School & Education,Soghaat,Superstore,Women's Fashion
0,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201711,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
201712,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
201713,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
201714,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False


In [23]:
frequent_itemsets

,support,itemsets


In [19]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

ValueError: The input DataFrame `df` containing the frequent itemsets is empty.